### LLM Chain

In [1]:
import os

ollama_base_url = os.environ["OLLAMA_BASE_URL"]

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2", base_url=ollama_base_url)

In [3]:
llm.invoke("how can langsmith help with testing?")

"\nLangsmith is a language model developed by Google that can be used for a variety of natural language processing tasks, including testing. Here are some ways Langsmith can help with testing:\n\n1. Automated Testing: Langsmith can be used to automate testing processes by generating test cases and evaluating the output of your application. This can save time and resources compared to manual testing methods.\n2. Regression Testing: Langsmith can be used to generate test cases for regression testing, which involves verifying that changes made to an application have not introduced new bugs or issues.\n3. Equivalence Testing: Langsmith can be used to generate test cases that check whether two different implementations of the same functionality are equivalent in terms of behavior and output.\n4. Exploratory Testing: Langsmith can be used to generate test cases that explore the boundaries of an application's functionality, identifying edge cases and uncovering unexpected behavior.\n5. Test D

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are world class technical documentation writer."),
        ("user", "{input}"),
    ]
)

In [5]:
chain = prompt | llm

In [6]:
response = chain.invoke({"input": "how can langsmith help with testing?"})
print(response)


As a world-class technical documentation writer, I'm glad you asked! Langsmith is not just a tool for creating and managing documentation, but it can also be leveraged to streamline testing processes. Here are some ways Langsmith can help with testing:

1. Automated Testing: Langsmith allows you to create automated tests for your application, using a variety of programming languages and test frameworks. This can help you identify issues early on in the development cycle, reducing the likelihood of errors and bugs making it to production.
2. Documentation Integration: With Langsmith, you can integrate documentation directly into your testing process. For example, you can create test cases that include expected outputs or behavior based on the documentation, ensuring that your tests are comprehensive and accurate.
3. Collaborative Testing: Langsmith enables real-time collaboration for testing, allowing multiple team members to work together on a single test case. This promotes coordinat

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser

In [9]:
response = chain.invoke({"input": "how can langsmith help with testing?"})
print(response)


Ah, an excellent question! As a technical documentation writer, I'm glad you asked about Langsmith's capabilities in testing. You see, Langsmith is not just a language model, but a comprehensive tool that can assist with various aspects of software development, including testing.

Langsmith's AI-powered engine can help with testing by:

1. Automating Testing: Langsmith can automate repetitive tasks such as data entry, test case creation, and test execution. This frees up your time to focus on more critical tasks, like ensuring the software is functioning correctly and meets user expectations.
2. Identifying Bugs: Langsmith's natural language processing (NLP) capabilities allow it to analyze vast amounts of data, identify patterns, and detect anomalies. This can help you catch bugs and errors earlier in the development process, reducing the likelihood of critical issues slipping through the cracks.
3. Generating Test Cases: Langsmith can create test cases based on your requirements and

### Retrieval Chain

In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [11]:
print(docs[0].page_content)






LangSmith User Guide | 🦜️🛠️ LangSmith







Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging​When developing new LL

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama2", base_url=ollama_base_url)

In [13]:
embeddings

OllamaEmbeddings(base_url='http://ollama:11434', model='llama2', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, model_kwargs=None)

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [15]:
documents

[Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}),
 Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the bread

In [16]:
vector

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)

document_chain = create_stuff_documents_chain(llm, prompt)

In [18]:
from langchain_core.documents import Document

document_chain.invoke(
    {
        "input": "how can langsmith help with testing?",
        "context": [
            Document(page_content="langsmith can let you visualize test results")
        ],
    }
)

'Based on the provided context, Langsmith can help with testing by allowing users to visualize their test results. This suggests that Langsmith is a tool that provides a way to display and analyze data from tests, which can be useful for understanding the outcome of tests and identifying patterns or trends in the data.'

In [19]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

 Based on the provided context, LangSmith can help with testing in several ways:

1. Creating test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
2. Tracking regressions: LangSmith makes it easy to run custom evaluations (both LLM and heuristic based) to score test results. This helps developers identify regressions in their LLM applications and make necessary improvements.
3. Comparison view: LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application. This allows developers to quickly identify which variant is performing better.
4. Beta testing: LangSmith enables developers to collect more data on how their LLM applications are performing in real-world scenarios. This helps d

### Conversation Retrieval

In [21]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation",
        ),
    ]
)

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [22]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage("Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!"),
]

retriever_chain.invoke({"chat_history": chat_history, "input": "Tell me how"})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications,

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [24]:
chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!"),
]

response = retrieval_chain.invoke({"chat_history": chat_history, "input": "Tell me how"})
print(response["answer"])

Of course! LangSmith is designed to support the testing and evaluation of LLM applications at various stages of the development lifecycle. Here are some ways LangSmith can help you test your LLM applications:

1. Prototyping: LangSmith allows you to rapidly experiment with different prompts, model types, and retrieval strategies. You can use the platform's debugging tools to identify and debug any issues that arise during testing.
2. Debugging: When developing new LLM applications, LangSmith provides native rendering of chat messages, functions, and retrieve documents. This makes it easier to understand how the model is performing and where it may be failing.
3. Initial Test Set: LangSmith allows you to create datasets of inputs and reference outputs, which can be used to run tests on your LLM applications. You can also upload test cases in bulk, create them on the fly, or export them from application traces.
4. Comparison View: LangSmith provides a user-friendly comparison view for te